# हॉटेल बुकिंगसाठी प्राधान्य सदस्य मिडलवेअर

या नोटबुकमध्ये Microsoft Agent Framework वापरून **फंक्शन-आधारित मिडलवेअर** कसे कार्य करते ते दाखवले आहे. आम्ही अटींवर आधारित कार्यप्रवाहाच्या उदाहरणावर आधारित काम करत आहोत आणि प्राधान्य सदस्यांना विशेष सुविधा देणारी मिडलवेअर स्तर जोडत आहोत.

## तुम्ही काय शिकाल:
1. **फंक्शन-आधारित मिडलवेअर**: फंक्शनचे परिणाम अडवणे आणि बदलणे
2. **कंटेक्स्ट ऍक्सेस**: `context.result` वाचणे आणि अंमलबजावणीनंतर बदलणे
3. **व्यवसाय तर्काची अंमलबजावणी**: प्राधान्य सदस्य लाभ
4. **परिणाम ओव्हरराइड**: वापरकर्त्याच्या स्थितीनुसार फंक्शनचे परिणाम बदलणे
5. **समान कार्यप्रवाह, वेगवेगळे परिणाम**: मिडलवेअरद्वारे वर्तन बदल

## मिडलवेअरसह कार्यप्रवाहाची रचना:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## अटींवर आधारित कार्यप्रवाहापेक्षा मुख्य फरक:

**मिडलवेअरशिवाय** (14-conditional-workflow.ipynb):
- पॅरिसमध्ये खोल्या नाहीत → alternative_agent कडे रूट करा

**मिडलवेअर सह** (या नोटबुकमध्ये):
- नियमित वापरकर्ता + पॅरिस → खोल्या नाहीत → alternative_agent कडे रूट करा
- प्राधान्य वापरकर्ता + पॅरिस → 🌟 मिडलवेअर ओव्हरराइड! → उपलब्ध → booking_agent कडे रूट करा

## पूर्वअट:
- Microsoft Agent Framework स्थापित केलेला असणे
- अटींवर आधारित कार्यप्रवाह समजणे (14-conditional-workflow.ipynb पहा)
- GitHub टोकन किंवा OpenAI API की
- मिडलवेअर पॅटर्न्सची मूलभूत समज


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## चरण 1: संरचित आउटपुटसाठी Pydantic मॉडेल्स परिभाषित करा

हे मॉडेल्स एजंट्स परत करणार असलेल्या **स्कीमा** परिभाषित करतात. आम्ही `priority_override` फील्ड जोडली आहे जेव्हा मिडलवेअर उपलब्धता परिणाम बदलते तेव्हा त्याचा मागोवा ठेवण्यासाठी.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## चरण 2: प्राधान्य सदस्य डेटाबेस परिभाषित करा

या डेमोसाठी, आपण प्राधान्य सदस्यत्व डेटाबेसची नक्कल करणार आहोत. उत्पादनामध्ये, हे वास्तविक डेटाबेस किंवा API क्वेरी करेल.

**प्राधान्य सदस्य:**
- `alice@example.com` - व्हीआयपी सदस्य
- `bob@example.com` - प्रीमियम सदस्य  
- `priority_user` - चाचणी खाते


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## चरण 3: हॉटेल बुकिंग टूल तयार करा

पूर्वीच्या अटींवर आधारित कार्यप्रवाहासारखेच, परंतु आता ते मिडलवेअरद्वारे अडवले जाईल!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण 4: 🌟 प्राधान्य तपासणी मिडलवेअर तयार करा (मुख्य वैशिष्ट्य!)

हे या नोटबुकचे **मुख्य कार्य** आहे. मिडलवेअर:

1. **हॉटेल बुकिंग फंक्शन कॉल अडवते**
2. `next(context)` कॉल करून फंक्शन सामान्यपणे **अंमलात आणते**
3. `context.result` मधील निकाल **तपासते**
4. जर वापरकर्ता प्राधान्य असलेला असेल आणि उपलब्ध खोल्या नसतील तर निकाल **ओव्हरराइड करते**
5. **सुधारित निकाल एजंटला परत देते**

**महत्त्वाचा नमुना:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## चरण ५: रूटिंगसाठी अटींच्या फंक्शन्स परिभाषित करा

सशर्त कार्यप्रवाहासारख्या अटींच्या फंक्शन्स - ते संरचित आउटपुट तपासून रूटिंग निश्चित करतात.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## चरण 6: कस्टम डिस्प्ले एक्झिक्युटर तयार करा

पूर्वीप्रमाणेच एक्झिक्युटर - अंतिम वर्कफ्लो आउटपुट दर्शवतो.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## चरण ७: पर्यावरणीय चल (Environment Variables) लोड करा

LLM क्लायंट (GitHub Models किंवा OpenAI) कॉन्फिगर करा.


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## चरण 8: मिडलवेअरसह AI एजंट तयार करा

**महत्त्वाचा फरक:** उपलब्धता_agent तयार करताना, आम्ही `middleware` पॅरामीटर पास करतो!

याद्वारेच आम्ही priority_check_middleware एजंटच्या फंक्शन कॉल पाईपलाइनमध्ये समाविष्ट करतो.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## चरण 9: कार्यप्रवाह तयार करा

पूर्वीप्रमाणेच कार्यप्रवाहाची रचना - उपलब्धतेवर आधारित सशर्त मार्गक्रमण.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## चरण 10: चाचणी प्रकरण 1 - पॅरिसमधील नियमित वापरकर्ता (कोणताही ओव्हरराइड नाही)

एक नियमित वापरकर्ता पॅरिस बुक करण्याचा प्रयत्न करतो → कोणतीही खोली उपलब्ध नाही → पर्यायी एजंटकडे मार्गक्रमण


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## चरण 11: चाचणी प्रकरण 2 - 🌟 प्राधान्य वापरकर्ता पॅरिसमध्ये (ओव्हरराइडसह!)

एक प्राधान्य सदस्य पॅरिस बुक करण्याचा प्रयत्न करतो → सुरुवातीला कोणतीही खोली उपलब्ध नाही → 🌟 मिडलवेअर ओव्हरराइड करते! → booking_agent कडे रूट होते

**हे मिडलवेअरच्या सामर्थ्याचे मुख्य प्रदर्शन आहे!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## चरण 12: चाचणी प्रकरण 3 - स्टॉकहोममधील प्राधान्य वापरकर्ता (आधीपासून उपलब्ध)

प्राधान्य वापरकर्ता स्टॉकहोम प्रयत्न करतो → खोल्या उपलब्ध आहेत → कोणत्याही ओव्हरराइडची गरज नाही → booking_agent कडे रूट होते

हे दाखवते की middleware फक्त आवश्यक असल्यावरच कार्य करते!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## मुख्य मुद्दे आणि मिडलवेअर संकल्पना

### ✅ तुम्ही काय शिकलात:

#### **1. फंक्शन-आधारित मिडलवेअर पॅटर्न**

मिडलवेअर एक साधी असिंक्रोनस फंक्शन वापरून फंक्शन कॉल अडवते:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. कॉन्टेक्स्ट ऍक्सेस आणि परिणाम ओव्हरराइड**

- `context.function` - कॉल केलेल्या फंक्शनला ऍक्सेस करा
- `context.arguments` - फंक्शनचे आर्ग्युमेंट्स वाचा
- `context.kwargs` - अतिरिक्त पॅरामीटर्स ऍक्सेस करा
- `await next(context)` - फंक्शन कार्यान्वित करा
- `context.result` - फंक्शनचा आउटपुट वाचा/सुधारा

#### **3. व्यवसायिक लॉजिक अंमलबजावणी**

आमचे मिडलवेअर प्राधान्य सदस्य लाभ अंमलात आणते:
- **सामान्य वापरकर्ते**: कोणतेही बदल नाहीत, सामान्य कार्यप्रवाह
- **प्राधान्य वापरकर्ते**: "कोणतीही उपलब्धता नाही" → "उपलब्ध" असे ओव्हरराइड करा
- **शर्ती आधारित लॉजिक**: फक्त आवश्यक असल्यास ओव्हरराइड करते

#### **4. समान कार्यप्रवाह, वेगवेगळे परिणाम**

मिडलवेअरची ताकद:
- ✅ कार्यप्रवाह संरचनेत कोणतेही बदल नाहीत
- ✅ टूल फंक्शनमध्ये कोणतेही बदल नाहीत
- ✅ शर्ती आधारित रूटिंग लॉजिकमध्ये कोणतेही बदल नाहीत
- ✅ फक्त मिडलवेअर → वेगळा वर्तन!

### 🚀 वास्तविक-जगातील उपयोग:

1. **VIP/प्रिमियम वैशिष्ट्ये**
   - प्रिमियम वापरकर्त्यांसाठी दर मर्यादा ओव्हरराइड करा
   - संसाधनांवर प्राधान्य ऍक्सेस प्रदान करा
   - प्रिमियम वैशिष्ट्ये डायनॅमिकरी अनलॉक करा

2. **A/B टेस्टिंग**
   - वापरकर्त्यांना वेगवेगळ्या अंमलबजावणीकडे रूट करा
   - विशिष्ट वापरकर्त्यांसह नवीन वैशिष्ट्ये चाचणी करा
   - वैशिष्ट्ये हळूहळू रोल आउट करा

3. **सुरक्षा आणि अनुपालन**
   - फंक्शन कॉल्सचे ऑडिट करा
   - संवेदनशील ऑपरेशन्स ब्लॉक करा
   - व्यवसाय नियम लागू करा

4. **कामगिरी ऑप्टिमायझेशन**
   - विशिष्ट वापरकर्त्यांसाठी परिणाम कॅश करा
   - शक्य असल्यास महागड्या ऑपरेशन्स टाळा
   - डायनॅमिक संसाधन वाटप करा

5. **त्रुटी हाताळणी आणि पुन्हा प्रयत्न**
   - त्रुटी सौम्यपणे पकडा आणि हाताळा
   - पुन्हा प्रयत्न करण्याचे लॉजिक अंमलात आणा
   - पर्यायी अंमलबजावणीवर फॉलबॅक करा

6. **लॉगिंग आणि मॉनिटरिंग**
   - फंक्शन कार्यान्वयन वेळा ट्रॅक करा
   - पॅरामीटर्स आणि परिणाम लॉग करा
   - वापर पॅटर्न मॉनिटर करा

### 🔑 डेकोरेटर्सशी मुख्य फरक:

| वैशिष्ट्य | डेकोरेटर | मिडलवेअर |
|-----------|-----------|------------|
| **व्याप्ती** | एकल फंक्शन | एजंटमधील सर्व फंक्शन्स |
| **लवचिकता** | परिभाषेवर निश्चित | रनटाइमवर डायनॅमिक |
| **कॉन्टेक्स्ट** | मर्यादित | पूर्ण एजंट कॉन्टेक्स्ट |
| **संयोजन** | एकाधिक डेकोरेटर्स | मिडलवेअर पाइपलाइन |
| **एजंट-जाणीव** | नाही | होय (एजंट स्टेट ऍक्सेस करा) |

### 📚 मिडलवेअर कधी वापरावे:

✅ **मिडलवेअर वापरा जेव्हा:**
- तुम्हाला वापरकर्ता/सेशन स्टेटवर आधारित वर्तन बदलायचे आहे
- तुम्हाला अनेक फंक्शन्सवर लॉजिक लागू करायचे आहे
- तुम्हाला एजंट-स्तरीय कॉन्टेक्स्ट ऍक्सेस करायचा आहे
- तुम्ही क्रॉस-कटिंग चिंता (लॉगिंग, ऑथ, इ.) अंमलात आणत आहात

❌ **मिडलवेअर वापरू नका जेव्हा:**
- साधी इनपुट व्हॅलिडेशन (Pydantic वापरा)
- फंक्शन-विशिष्ट लॉजिक (फंक्शनमध्ये ठेवा)
- एक-वेळ बदल (फक्त फंक्शन बदला)

### 🎓 प्रगत पॅटर्न:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 संबंधित संकल्पना:

- **एजंट मिडलवेअर**: agent.run() कॉल्स अडवते
- **फंक्शन मिडलवेअर**: टूल फंक्शन कॉल्स अडवते (आम्ही वापरलेले!)
- **मिडलवेअर पाइपलाइन**: क्रमाने कार्यान्वित होणाऱ्या मिडलवेअरची साखळी
- **कॉन्टेक्स्ट प्रोपगेशन**: मिडलवेअर साखळीमधून स्टेट पास करा



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
